In [178]:
import numpy as np
from PIL import Image

In [179]:
def transform(targetX,targetY):
    temp = np.array([targetX, targetY, targetX*targetY, 1])
    x, y = solveEquations()
    originalX = np.sum(np.matmul(temp, x))
    originalY = np.sum(np.matmul(temp, y))
    return originalX,originalY

In [180]:
def solveEquations():
    if not cache:
        a = np.mat('0,0,0,1; {},0,0,1,; 0,{},0,1; {},{},{},1'.format(width, height, width, height, width * height))     # Reference by x = ax' + by' + cx'y' + d
        b = np.mat('{},{},{},{}'.format(coordinatesOfX[0], coordinatesOfX[1], coordinatesOfX[2], coordinatesOfX[3])).T  # Reference by y = ex' + fy' + gx'y' + h
        c = a
        d = np.mat('{},{},{},{}'.format(coordinatesOfY[0], coordinatesOfY[1], coordinatesOfY[2], coordinatesOfY[3])).T  # Reference by y = ex' + fy' + gx'y' + h
        x = np.linalg.solve(a,b)
        y = np.linalg.solve(c,d)
        cache['x'] = x
        cache['y'] = y
    return cache['x'], cache['y']

In [190]:
def execBilinearInterpolation(x,y, targetX, targetY):
    # formula f2(x,y) = (1-a)(1-b)gs(l,k)+ a(1-b)gs(l+1,k) + (1-a)(1-b)bgs(l,k+1)+ abgs(l+1,k+1)
    intX = int(x)
    intY = int(y)
    # d2 means distanceTo 
    d2TopLeft = (intX + 1 - x)* (intY + 1 - y) # alias d2TL
    d2TopRight = (x - intX)* (intY + 1 - y)    # alias d2TR
    d2BottomLeft = (x - intX)* (y - intY)      # alias d2BL
    d2BottomRight = (intX + 1 - x)* (y - intY) # alias d2BR

    d2TLR, d2TLG, d2TLB = pixels[intX,y]
    d2TRR, d2TRG, d2TRB = pixels[intX + 1,y]
    d2BLR, d2BLG, d2BLB = pixels[intX + 1,intY +1]
    d2BRR, d2BRG, d2BRB = pixels[intX, intY + 1]
    targetImagePixels[targetX, targetY] = \
    int((d2TopLeft * d2TLR + d2TopRight * d2TRR + d2BottomLeft * d2BLR + d2BottomRight * d2BRR)), \
    int((d2TopLeft * d2TLG + d2TopRight * d2TRG + d2BottomLeft * d2BLG + d2BottomRight * d2BRG)), \
    int((d2TopLeft * d2TLB + d2TopRight * d2TRB + d2BottomLeft * d2BLB + d2BottomRight * d2BRB))

In [192]:
# Image shape is 1108 x 1478
#sourceImage = cv2.imread('dante.jpg')
#(sourceImageHeight, sourceImageWidth, sourceImageChannels) = sourceImage.shape
image = Image.open('dante.jpg', "r")
sourceImageWidth, sourceImageHeight = image.size
pixels = image.load()

# Set up the target image size(300 x 400)
height = 400
width = 300
targetImage = Image.new("RGB", (width,height))
targetImagePixels = targetImage.load()

# coordinate of an item which wants to do the geometric transformation
# the coordinate are in order: top left, top right, bottom left, bottom right
coordinates = np.array([[320, 494], [750, 487], [278, 863], [833, 845]])
coordinatesOfX = np.array([coordinates[0][0], coordinates[1][0], coordinates[2][0], coordinates[3][0]])
coordinatesOfY = np.array([coordinates[0][1], coordinates[1][1], coordinates[2][1], coordinates[3][1]])

# cache for equations
cache = {}

for y in range(0, height):
    for x in range(0, width):
        originalX, originalY = transform(x, y)
        execBilinearInterpolation(originalX, originalY, x, y)
targetImage.save('newDante.jpg')